In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import re
import datetime
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import norm, probplot
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from arch.unitroot import VarianceRatio
import scipy.stats as stats
from statsmodels.tsa.stattools import adfuller 
from scipy.stats import genpareto
from sklearn.utils import resample
from scipy.optimize import minimize
from scipy.special import expit, logit
from sklearn.linear_model import LogisticRegression
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from scipy.optimize import minimize
from scipy.stats import genpareto
from scipy.optimize import differential_evolution
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from scipy.optimize import differential_evolution
from scipy.stats import gumbel_r
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import RFE
import ta
import shap
from xgboost.callback import EarlyStopping
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.preprocessing import label_binarize
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

### Import Data

In [2]:
btc = pd.read_csv(r'/Users/xiaoyuxu/Desktop/Dissertation-docts/bitcoin_2017_to_2023.csv').iloc[::-1]
btc.set_index(btc.columns[0], inplace=True)
btc = btc[~btc.index.duplicated(keep='first')]
btc.index = pd.to_datetime(btc.index)

btc_min_return = pd.read_csv('btc_min_return_selected.csv')
btc_min_return.index =pd.to_datetime(btc_min_return['timestamp'])
btc_min_return.drop(columns=['timestamp'],inplace=True)

append_features=btc.columns.to_list()
append_features.remove('close')
append_features.remove('volume')

btc_min_return=pd.merge(btc[append_features], btc_min_return, left_index=True, right_index=True, how='inner')

columns_for_normalization = ['volume', 'number_of_trades']

scaler = StandardScaler()
btc_min_return.loc[:, columns_for_normalization] = scaler.fit_transform(btc_min_return[columns_for_normalization])
btc_min_return.drop(columns=['taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume','quote_asset_volume'],inplace=True)

KeyError: "['number_of_trades'] not in index"

### Additional Feature: Realized Semi-Variance

- Realized Semi-variance (RS): a new measure of the variation of asset prices based on high frequency data. We shall derive its limiting properties, relating it to quadratic variation and nagetive jumps. 

- Mathematically, RV estimates the ex-post variance of log asset prices Y over a fixed time period:

$RV = \sum_{j=1}^n (Y_{t_j} - Y_{t_{j-1}})^2$;

s.t. $(Y_t)$ is a semi-martingale:

$Y_t = M_t + A_t = \int_0^t \sigma_s d W_s + \int_0^t a_s d s$;

where $M$ is the local Martingale and $A$ is the predictable finite variation process. 

This estimator converges as we have more and more data in that interval to the quadratic variation:

$[Y]_1 = \lim_{n \rightarrow \infty} \sum_{j=1}^n (Y_{t_j} - Y_{t_{j-1}})^2$;

and

$[Y]_t = \int_0^t \sigma_s d W_s + \int_0^t a_s d s \quad \text{and} \quad d [Y]_t = \sigma_t^2 d t$.

i.e. for a Brownian semi-martingale the QV process tells us everthing we can know about the ex-post variation of $Y$.

However, if there are jumps:

$Y_t = \int_0^t \sigma_s d W_s + \int_0^t a_s d s + J_t$;

and

$[Y]_t = \int_0^t \sigma_s^2 d s + \sum_{s \leq t} (\triangle Y_s)^2$

with $ \triangle Y_t = Y_t - Y_{t-1}$.

The downside realized semi-variance is defined as:

$RS^- = \sum_{j=1}^n (Y_{t_j} - Y_{t_{j-1}})^2 \mathbb{1}_{Y_{t_j} - Y_{t_{j-1}} \leq 0}$;

s.t.

$RS^- \rightarrow \frac{1}{2} \int_0^1 \sigma_s^2 d s + \sum_{s \leq 1} (\triangle Y_s)^2 \mathbb{1}_{\triangle Y_s \leq 0}$.

And of course: $RV = RS^- + RS^+$.

Practiacally, we choose to measure returns using log-scale and multiplied by 100 and measure realized volatility based upon retursn calculated every 15 trades. The argument here to add $RS^-$ into the set of features is due to the argument that: negative shocks tend to increase future volatility more than positive shocks of the same magnitude - presumably because the high frequency falls increase the risk premium. (Woth of run the t-test to check the significance of $RS^-$ and lagged $RS^-$ as the evaluation metric of the prediction power.)







### Additional Feature: Realized Bipower Variation 

Mathematically, the realized bipower variation is defined as:

$BPV = \frac{1}{\mu_1^2} \sum_{j=2}^{t_j \leq t} |Y_{t_j} - Y_{t_{j-1}}| |Y_{t_{j-1}} - Y_{t_{j-2}}| \rightarrow \{Y\}_t^{[1,1]} = \int_0^t \sigma_s^2 d s$;

where 

$\mu_1 = \mathbb{E} |U| \quad U \sim \mathcal{N}(0,1)$.

i.e.

$RV - BPV \rightarrow [Y]_1 - \{Y\}^{[1,1]} = \sum_{s \leq t} (\triangle Y_s)^2$.

In [16]:
# Function to calculate downside realized semi-variance (RS_neg)
def downside_realized_semi_variance(returns):
    returns = returns*100
    negative_returns = returns[returns < 0]
    if len(negative_returns) == 0:
        return 0
    squared_negative_returns = negative_returns ** 2
    RS_neg = squared_negative_returns.sum() / len(negative_returns)
    return RS_neg

window_size = 15
btc_min_return['RS_neg'] = btc_min_return['return'].rolling(window=window_size).apply(downside_realized_semi_variance, raw=True)

In [17]:
def realized_bipower_variation(returns):
    returns = returns*100
    abs_returns = np.abs(returns)
    product_sum = np.sum(abs_returns[:-1] * abs_returns[1:])
    BPV = (np.pi / 2) * product_sum
    return BPV

btc_min_return['BPV'] = btc_min_return['return'].rolling(window=window_size).apply(realized_bipower_variation, raw=True)

In [18]:
btc_min_return['RS'] = (btc_min_return['return']*100).rolling(window=15).sum()

In [19]:
btc_min_return['JP'] = btc_min_return['RS'] - btc_min_return['BPV']

### Generate Basic Features

- Exponential Weighted Moving Average (EWMA)

$EWMA_t = \alpha \cdot r_t + (1-\alpha) \cdot EWMA_{t-1}$.

- Moving Average Convergence Divergence (MACD)

$MACD_t = EMA_{12,t} - EMA_{26, t} \quad Signal_t = EMA_{9, MACD_t}$.

- Relative Strength Index (RSI)

RSI is a momentum oscillator that measures the speed and change of price movements and used to identify overbought and oversold.

$RS = \frac{AG}{AL} \quad RSI = 100 - \frac{100}{1+RS}$;

where $AG$ and $AL$ are the average gain and loss over a specified period, typically $14$.

- Bollinger Bands

$\text{Middle Band}_t = SMA_{20,t} \quad \text{Upper Band}_t = \text{Middle Band}_t + 2 \sigma_t$;

where $\sigma_t$ is the std of Middle Band.

- Average True Range (ATR)

ATR is typically derived from the 14-day period, and it measures the degree of price volatility.

$TR_1 = \text{High}_t - \text{Low}_t \quad TR_2 = |\text{High}_t - \text{Close}_{t-1}| \quad TR_3 = |\text{Low}_t - \text{Close}_{t-1}|$;

then,

$TR_t = \max \{TR_1, TR_2, TR_3\} \quad ART = SMA_{14, TR}$.


In [21]:
# Create rolling statistics
btc_min_return['rolling_mean_return_15min'] = btc_min_return['return'].rolling(window=15).mean()
btc_min_return['rolling_std_return_15min'] = btc_min_return['return'].rolling(window=15).std()
btc_min_return['rolling_mean_volume_15min'] = btc_min_return['volume'].rolling(window=15).mean()
btc_min_return['rolling_std_volume_15min'] = btc_min_return['volume'].rolling(window=15).std()

# Create lag features (1, 5, 10, 15, 30 minutes)
lags = [1, 5, 10, 15]
for lag in lags:
    btc_min_return[f'return_lag_{lag}min'] = btc_min_return['return'].shift(lag)

# Interaction features
btc_min_return['return_volume_interaction'] = btc_min_return['return'] * btc_min_return['volume']

# Exponential moving averages
btc_min_return['ema_return_5min'] = btc_min_return['return'].ewm(span=5, adjust=False).mean()

# Volatility measures
btc_min_return['hist_vol_5min'] = btc_min_return['return'].rolling(window=5).std()

# Technical Indicators 
# Moving Average Convergence Divergence (MACD)
btc_min_return['macd'] = ta.trend.MACD(btc_min_return['return']).macd()
btc_min_return['macd_signal'] = ta.trend.MACD(btc_min_return['return']).macd_signal()

# Relative Strength Index (RSI)
btc_min_return['rsi'] = ta.momentum.RSIIndicator(btc_min_return['return']).rsi()

# Bollinger Bands
bollinger = ta.volatility.BollingerBands(btc_min_return['return'])
btc_min_return['bollinger_hband'] = bollinger.bollinger_hband()
btc_min_return['bollinger_lband'] = bollinger.bollinger_lband()

# Average True Range (ATR)
atr = ta.volatility.AverageTrueRange(
    high=btc_min_return['close'], 
    low=btc_min_return['low'], 
    close=btc_min_return['close']
)
btc_min_return['atr'] = atr.average_true_range()

btc_min_return=btc_min_return.dropna()

In [22]:
btc_min_return.to_csv('btc_min_return_selected.csv', index=True)